In [8]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import os

def get_access_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {"Content-Type" : "application/x-www-form-urlencoded"}
    data = {"grant_type" : "client_credentials", "client_id" : os.getenv("SPOTIFY_CLIENT_ID"), "client_secret" : os.getenv("SPOTIFY_CLIENT_SECRET")}
    # OBS OBS OBS!!! This function will not work unless os.getenv("SPOTIFY_CLIENT_ID") and os.getenv("SPOTIFY_CLIENT_SECRET") is changed 
    # into an actual clien_id, and actual_secret for spotify web api. This can easily be made by following the link: https://developer.spotify.com/documentation/web-api
    
    response = requests.post(url, headers=headers, data=data)

    token_data = response.json()
    access_token = token_data.get('access_token')
    return access_token


def call_api(url, headers, offset, genre):
    params = {"q" : f"genre:{genre}", "type" : "track", "limit" : 50, "offset" : offset}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data.get('tracks').get('items')


def get_tracks(access_token):
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization" : f"Bearer {access_token}"}

    tracks = []
    popular_genres= ['pop', 'hip-hop', 'rap', 'rock', 'electronic', 'dance', 'latin', 'indie', 'r&b', 'soul', 'country', 'jazz']

    for genre in popular_genres:
        for i in range(20):
            list_of_tracks = call_api(url, headers, i*50, genre)
            for track in list_of_tracks:
                tracks.append(track)
    return tracks


def get_albums_artists(tracks):
    albums = []
    albums_id = []
    artists = []

    for track in tracks: 
        albums.append(track.get('album'))
        albums_id.append(track.get('album').get('id'))
        artists.append(track.get('artists'))

    return albums, albums_id, artists

def get_artists_id(artists):
    artist_id_lists = []

    for artist_or_artists in artists:
        id_list=[]
        for artist in artist_or_artists:
            id_list.append(artist.get('id'))        
        artist_id_lists.append(id_list)

    return artist_id_lists

def get_df_tracks(tracks, albums_id, artist_id_lists):
    df_tracks = pd.DataFrame(tracks)
    df_tracks.insert(0,'album_id', albums_id)
    df_tracks.insert(2,'artists_id', artist_id_lists)
    df_tracks = df_tracks.drop_duplicates("id")
    return df_tracks

def get_df_albums(albums):
    df_albums = pd.DataFrame(albums)
    df_albums = df_albums.drop_duplicates("id")
    return df_albums

def main():
    access_token = get_access_token()
    tracks = get_tracks(access_token)
    albums, albums_id, artists = get_albums_artists(tracks)
    artist_id_lists = get_artists_id(artists)
    df_tracks = get_df_tracks(tracks, albums_id, artist_id_lists)
    df_albums = get_df_albums(albums)
    #Remove the hashtags to errase
    #df_tracks.to_csv("tracks.csv", index = False) 
    #df_albums.to_csv("albums.csv", index = False)

main()